In [1]:
import random,math,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn import svm
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [3]:
def case1(move):
    return 1 if random.uniform(-0.1,0.9)<move else 0
def case2(one_bin):
    if random.uniform(-0.1,0.9)<math.tanh(int(one_bin)):
        if one_bin==1:
            return 0
        else:return 1
    else:return one_bin
def exchange_binary(binary,score,alpha,beta,gamma):
    al_binary=binary
    e=0.05*random.uniform(0,1)
    movement=beta*math.exp(-gamma*score**2)*score + alpha*e
    if random.uniform(0,1) < movement:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case1(move)
    else:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case2(move)
    return al_binary

In [4]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [5]:
def BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {tuple([0]*dim):float("-inf")}
    gens=random_search(n,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
    for it in range(max_iter):
        for i,x in enumerate(gens):
            for j,y in enumerate(gens):
                if gens_dict[tuple(y)] < gens_dict[tuple(x)]:
                    gens[j]=exchange_binary(y,gens_dict[tuple(y)],alpha,beta,gamma)
                gen = gens[j]
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gens[j])
                    gens_dict[tuple(gen)]=score
                if score > global_best:
                    global_best=score
                    global_position=dc(gen)
    return global_position,global_position.count(1)


In [6]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([svm.LinearSVC().fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [7]:
with open("/home/shubbham28/Downloads/bc/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/bc/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [18]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.8601398601398602
1  000010001000000000011000001001  6  0.955245
2  000001100100000000011000000001  6  0.962238
3  000000000000000000101000010001  4  0.952448
4  000000000000000000001000001000  2  0.962238
5  000010000000000000001000001001  4  0.962238
6  000001001010001000001100100110  9  0.945455
7  000000000001000000001000001000  3  0.963636
8  110000000100000000001000010000  5  0.965035
9  010001110000000000001000000000  5  0.924476
10  010000011000000000001000001000  5  0.969231
11  000000000000001100001100001100  6  0.966434
12  000000000000000001001100001000  4  0.949650
13  000000000011000010001000101000  6  0.963636
14  000010000000000000001100001001  5  0.937063
15  000000100101000010011000000110  8  0.959441
16  000001000000000110001100001010  7  0.963636
17  000000100010000000001100010000  5  0.946853
18  000000100000001010001000000010  5  0.959441
19  000001001000001001001000010010  7  0.951049
20  000000100000001000001100001100  6  0.969231
Final:   000000100000000000001